In [4]:
from dataload import get_dataloaders

train,test,val,info = get_dataloaders('DermaMNIST',batch_size=32)

Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/lotfimecharbat/.medmnist/dermamnist.npz


In [ ]:
from ofa.imagenet_classification.networks import ResNets
from ofa.imagenet_classification.elastic_nn.modules.dynamic_layers import DynamicMBConvLayer, DynamicConvLayer, DynamicLinearLayer
import torch

super_net = torch.hub.load('mit-han-lab/once-for-all', 'ofa_supernet_resnet50', pretrained=True)
n_classes = 40

# Modify classifier for the dataset
super_net.classifier = DynamicLinearLayer([super_net.classifier.linear.linear.in_features], n_classes)



Using cache found in /home/lotfimecharbat/.cache/torch/hub/mit-han-lab_once-for-all_master
Downloading: "https://huggingface.co/han-cai/ofa/resolve/main/ofa_supernet_resnet50" to .torch/ofa_nets/ofa_supernet_resnet50
/home/lotfimecharbat/Projects/MedNNS/ofa/model_zoo.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any 

In [7]:
# test supernet
x = torch.randn(1, 3, 244, 244)
logits = super_net(x)
print(logits.shape)

torch.Size([1, 40])


In [31]:
import random
# test subnetwork
def sample_active_subnet(supernet, sample_config):
    """
    Sample and set an active subnet configuration.
    """
    # sample expand ratio
    if isinstance(sample_config['e'][0], list):
        expand_setting = [random.choice(sample_config['e'][i]) for i in range(len(sample_config['e']))]
    else:
        expand_setting = [random.choice(sample_config['e']) for _ in range(len(supernet.blocks))]

    # sample depth
    depth_setting = []
    for stage_id in range(len(sample_config['d'])):
        depth_list = sample_config['d'][stage_id]
        depth_setting.append(random.choice(depth_list))

    # sample width_mult
    width_mult_index = [random.choice(sample_config['w'][i]) for i in range(len(sample_config['w']))]
    width_mult_setting = [
        list(range(len(supernet.input_stem[0].out_channel_list)))[width_mult_index[0]],
        list(range(len(supernet.input_stem[2].out_channel_list)))[width_mult_index[1]],
    ]
    for stage_id, block_idx in enumerate(supernet.grouped_block_index):
        stage_first_block = supernet.blocks[block_idx[0]]
        width_mult_setting.append(
            list(range(len(stage_first_block.out_channel_list)))[width_mult_index[stage_id+2]]
        )

    arch_config = {"d": depth_setting, "e": expand_setting, "w": width_mult_setting}
    supernet.set_active_subnet(**arch_config)
    return arch_config

sample_config={"d": [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]],
        "e": [0.35],
        "w": [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]]}

arch_config = sample_active_subnet(super_net, sample_config)
subnet=super_net.get_active_subnet()
# calculate n params
print(arch_config)
sum(p.numel() for p in subnet.parameters())- sum(p.numel() for p in super_net.classifier.parameters())

{'d': [2, 1, 0, 2, 1], 'e': [0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35], 'w': [1, 0, 2, 2, 1, 1]}


24502520